
# 🧠 Caso de Estudio: Optimización del Control de Calidad en una Línea de Producción  
**Minería de Datos - Ingeniería Industrial**  

Este notebook guía a los estudiantes en la **preparación y limpieza de datos** de un proceso productivo, abordando los puntos:

1️⃣ Carga de datos  
2️⃣ Identificación de valores faltantes  
3️⃣ Imputación de datos (media/mediana/moda)  
4️⃣ Detección y tratamiento de outliers  
5️⃣ Escalado o normalización  


In [ ]:

# ================================================
# (0) Librerías
# ================================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ================================================
# (1) CARGA DE DATOS
# ================================================
csv_path = "caso_estudio_calidad_produccion.csv"  # subir el archivo a Colab o ruta local
df = pd.read_csv(csv_path)

print("✅ Datos cargados. Vista general:")
display(df.head())
print("\nInfo del DataFrame:")
print(df.info())


In [ ]:

# ================================================
# (2) IDENTIFICACIÓN DE VALORES FALTANTES
# ================================================
print("\n✅ Valores faltantes por columna:")
print(df.isna().sum())


In [ ]:

# ================================================
# (3) IMPUTACIÓN DE VALORES FALTANTES
# ================================================
vars_numericas = ["presion_bar", "temperatura_c", "enfriamiento_s"]
var_categorica = "lote_defectuoso"

medianas = df[vars_numericas].median(numeric_only=True)

for col in vars_numericas:
    df[col] = df[col].fillna(medianas[col])

if df[var_categorica].isna().any():
    modo = df[var_categorica].mode(dropna=True)[0]
    df[var_categorica] = df[var_categorica].fillna(modo)

print("\n✅ Imputación completada. Medianas usadas:")
print(medianas)
display(df)


In [ ]:

# ================================================
# (4) DETECCIÓN Y TRATAMIENTO DE OUTLIERS
# ================================================
df_clean = df.copy()

# Regla de negocio: presión > 100 bar es imposible
umbral_presion_max = 100.0
mask_out_presion = df_clean["presion_bar"] > umbral_presion_max
df_clean.loc[mask_out_presion, "presion_bar"] = df_clean["presion_bar"].median()

# Método IQR para temperatura
Q1 = df_clean["temperatura_c"].quantile(0.25)
Q3 = df_clean["temperatura_c"].quantile(0.75)
IQR = Q3 - Q1
lim_inf = Q1 - 1.5 * IQR
lim_sup = Q3 + 1.5 * IQR

mask_out_temp = (df_clean["temperatura_c"] < lim_inf) | (df_clean["temperatura_c"] > lim_sup)
df_clean.loc[df_clean["temperatura_c"] < lim_inf, "temperatura_c"] = lim_inf
df_clean.loc[df_clean["temperatura_c"] > lim_sup, "temperatura_c"] = lim_sup

print("\n✅ Outliers tratados (presión y temperatura).")
display(df_clean)


In [ ]:

# ================================================
# (5) ESCALADO / NORMALIZACIÓN
# ================================================
scaler = MinMaxScaler()
df_scaled = df_clean.copy()
df_scaled[vars_numericas] = scaler.fit_transform(df_clean[vars_numericas])

print("\n✅ Escalado MinMax completado (rango [0,1]).")
display(df_scaled.describe())

# Guardar versiones limpias
df_clean.to_csv("calidad_limpio.csv", index=False)
df_scaled.to_csv("calidad_limpio_escalado.csv", index=False)

print("\nArchivos guardados: calidad_limpio.csv y calidad_limpio_escalado.csv")
